In [ ]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import os, pickle
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

from itertools import product

import sys, os

import trajectory as T                      # trajectory generation
import optimizer as O                       # stochastic gradient descent optimizer
import solver as S                          # MDP solver (value-iteration)
import plot as P



# LOADING THE DATA

In [ ]:
# print(len(data_df))
# print(len(X_df))
# print(len(y_df))
# print(len(data_non_normalized_df))

data_df = pd.read_pickle('data_df_pneumonia.pkl')
X_df = pd.read_pickle('X_df_pneumonia.pkl')
y_df = pd.read_pickle('y_df_pneumonia.pkl')
data_non_normalized_df = pd.read_pickle('data_non_normalized_df_pneumonia.pkl')
# note: data should have cluster assignments already

In [ ]:
data_non_normalized_df.columns

Index(['subject_id', 'Creatinine (serum)', 'Inspired O2 Fraction',
       'Urine output_ApacheIV', 'ALT', 'AST', 'Arterial Blood Pressure mean',
       'Arterial Blood Pressure diastolic', 'Arterial Blood Pressure systolic',
       'GcsScore_ApacheIV', 'Arterial O2 pressure', 'Heart Rate',
       'Temperature Celsius', 'Respiratory Rate',
       'O2 saturation pulseoxymetry', 'action', 'cluster'],
      dtype='object')

In [ ]:
# PARAMS:
smoothing_value = 1
discount_exp = 0.001
action_bound = 15


# Corresponding output data for training BC (corresponding treatments for each data point in X_df)

In [ ]:
y_df.head()

0    2.0
1    8.0
2    8.0
3    4.0
4    0.0
Name: action, dtype: float64

In [ ]:
data_df.head()

subject_id  Creatinine (serum)  \
stay_id  time_hour                                             
30002521 2162-05-03 10:00:00    13269859           -0.662381   
         2164-02-07 10:00:00    13269859           -0.662381   
         2164-02-08 08:00:00    13269859           -0.662381   
30011071 2176-08-18 03:00:00    11885477           -0.662381   
30028516 2179-06-28 18:00:00    15517908           -0.662381   

                              Inspired O2 Fraction  Urine output_ApacheIV  \
stay_id  time_hour                                                          
30002521 2162-05-03 10:00:00             -2.531506                    0.0   
         2164-02-07 10:00:00             -2.531506                    0.0   
         2164-02-08 08:00:00             -2.531506                    0.0   
30011071 2176-08-18 03:00:00             -2.531506                    0.0   
30028516 2179-06-28 18:00:00             -2.531506                    0.0   

                                   ALT      AST  Arterial Blood Pressure mean  \
stay_id  time_hour                                                              
30002521 2162-05-03 10:00:00 -0.142368 -0.14189                     -0.064513   
         2164-02-07 10:00:00 -0.142368 -0.14189                     -0.064513   
         2164-02-08 08:00:00 -0.142368 -0.14189                     -0.064513   
30011071 2176-08-18 03:00:00 -0.142368 -0.14189                     -0.064513   
30028516 2179-06-28 18:00:00 -0.142368 -0.14189                     -0.064513   

                              Arterial Blood Pressure diastolic  \
stay_id  time_hour                                                
30002521 2162-05-03 10:00:00                           0.136648   
         2164-02-07 10:00:00                           0.136648   
         2164-02-08 08:00:00                           0.136648   
30011071 2176-08-18 03:00:00                           0.136648   
30028516 2179-06-28 18:00:00                           0.136648   

                              Arterial Blood Pressure systolic  \
stay_id  time_hour                                               
30002521 2162-05-03 10:00:00                          0.035025   
         2164-02-07 10:00:00                          0.035025   
         2164-02-08 08:00:00                          0.035025   
30011071 2176-08-18 03:00:00                          0.035025   
30028516 2179-06-28 18:00:00                          0.035025   

                              GcsScore_ApacheIV  Arterial O2 pressure  \
stay_id  time_hour                                                      
30002521 2162-05-03 10:00:00                0.0              2.266593   
         2164-02-07 10:00:00                0.0              2.266593   
         2164-02-08 08:00:00                0.0              2.266593   
30011071 2176-08-18 03:00:00                0.0              2.266593   
30028516 2179-06-28 18:00:00                0.0              2.266593   

                              Heart Rate  Temperature Celsius  \
stay_id  time_hour                                              
30002521 2162-05-03 10:00:00    0.067084            -0.174096   
         2164-02-07 10:00:00    0.661191            -0.174096   
         2164-02-08 08:00:00    0.013074            -0.174096   
30011071 2176-08-18 03:00:00    0.337132            -0.174096   
30028516 2179-06-28 18:00:00    0.013074            -0.174096   

                              Respiratory Rate  O2 saturation pulseoxymetry  \
stay_id  time_hour                                                            
30002521 2162-05-03 10:00:00         -0.050875                     0.634716   
         2164-02-07 10:00:00          0.292859                     0.076779   
         2164-02-08 08:00:00         -0.910210                     0.634716   
30011071 2176-08-18 03:00:00          0.636593                     1.192653   
30028516 2179-06-28 18:00:00          0.464726                     1.192653   

                  

In [ ]:
data_non_normalized_df.head()

subject_id  Creatinine (serum)  \
stay_id  time_hour                                             
30002521 2162-05-03 10:00:00    13269859                 1.0   
         2164-02-07 10:00:00    13269859                 1.0   
         2164-02-08 08:00:00    13269859                 1.0   
30011071 2176-08-18 03:00:00    11885477                 1.0   
30028516 2179-06-28 18:00:00    15517908                 1.0   

                              Inspired O2 Fraction  Urine output_ApacheIV  \
stay_id  time_hour                                                          
30002521 2162-05-03 10:00:00                  0.21                   80.0   
         2164-02-07 10:00:00                  0.21                   80.0   
         2164-02-08 08:00:00                  0.21                   80.0   
30011071 2176-08-18 03:00:00                  0.21                   80.0   
30028516 2179-06-28 18:00:00                  0.21                   80.0   

                               ALT   AST  Arterial Blood Pressure mean  \
stay_id  time_hour                                                       
30002521 2162-05-03 10:00:00  34.0  40.0                          77.0   
         2164-02-07 10:00:00  34.0  40.0                          77.0   
         2164-02-08 08:00:00  34.0  40.0                          77.0   
30011071 2176-08-18 03:00:00  34.0  40.0                          77.0   
30028516 2179-06-28 18:00:00  34.0  40.0                          77.0   

                              Arterial Blood Pressure diastolic  \
stay_id  time_hour                                                
30002521 2162-05-03 10:00:00                               59.0   
         2164-02-07 10:00:00                               59.0   
         2164-02-08 08:00:00                               59.0   
30011071 2176-08-18 03:00:00                               59.0   
30028516 2179-06-28 18:00:00                               59.0   

                              Arterial Blood Pressure systolic  \
stay_id  time_hour                                               
30002521 2162-05-03 10:00:00                             118.0   
         2164-02-07 10:00:00                             118.0   
         2164-02-08 08:00:00                             118.0   
30011071 2176-08-18 03:00:00                             118.0   
30028516 2179-06-28 18:00:00                             118.0   

                              GcsScore_ApacheIV  Arterial O2 pressure  \
stay_id  time_hour                                                      
30002521 2162-05-03 10:00:00               11.0                 276.0   
         2164-02-07 10:00:00               11.0                 276.0   
         2164-02-08 08:00:00               11.0                 276.0   
30011071 2176-08-18 03:00:00               11.0                 276.0   
30028516 2179-06-28 18:00:00               11.0                 276.0   

                              Heart Rate  Temperature Celsius  \
stay_id  time_hour                                              
30002521 2162-05-03 10:00:00        86.0                 37.0   
         2164-02-07 10:00:00        97.0                 37.0   
         2164-02-08 08:00:00        85.0                 37.0   
30011071 2176-08-18 03:00:00        91.0                 37.0   
30028516 2179-06-28 18:00:00        85.0                 37.0   

                              Respiratory Rate  O2 saturation pulseoxymetry  \
stay_id  time_hour                                                            
30002521 2162-05-03 10:00:00              19.0                         98.0   
         2164-02-07 10:00:00              21.0                         96.0   
         2164-02-08 08:00:00              14.0                         98.0   
30011071 2176-08-18 03:00:00              23.0                        100.0   
30028516 2179-06-28 18:00:00              22.0                        100.0   

                              action  cluster  
stay_id  time_hour 

In [ ]:
CLUSTER_LIST = data_non_normalized_df['cluster']


In [ ]:
# Looking at the values counts for each cluster

np.unique(CLUSTER_LIST, return_counts = True)[1]

array([ 649, 7815,  161, 2438,  603,  435, 1259, 2400, 1263,   28, 1579,
        254, 1238,  434,  303,  657,  725,  500,  655,  493,  530,  505,
       2028,   96,  925,  290, 2168,  135,  937, 1859,  987,  110,  393,
        599,  238,  147,  274,  165,  225,  124,  397,   21,  200,   73,
        638,  300,  831,  643,  117,  500,  278,  196,  509,  251,  146,
         64,  266,  822,  169,  539,  135,  399,  424,  248,  378,  557,
         94,  148,  179,   81,  189,  343,  110,  248,   72,  133,  468,
        144,  556,  594,  120,  403,  219,  614,  107,  315,   44,   56,
        119,   42,  112,  392,  156,  284,   81,  142,   87,  289,  180,
         63])

# Converting the data into trajectories to input to an IRL algorithm Note this is the same format of trajectories we used for HW1 and HW2.

In [ ]:
unique_stay_ids = data_df.index.get_level_values('stay_id').unique()
unique_subject_ids = set(data_df['subject_id'])
trajectories = []

for person in unique_subject_ids:
  filtered_df = data_df[data_df['subject_id']==person].drop(columns=['subject_id'])
  unique_stays = filtered_df.index.get_level_values('stay_id').unique()
  for stay_id in unique_stays:
    states, actions = filtered_df.loc[stay_id]['cluster'], filtered_df.loc[stay_id]['action']
    # print(len(states))

    trajectory = []
    if len(states)<2:
      continue
    else:
      for i in range(len(states) - 1):
        trajectory.append((states[i], int(actions[i]), states[i+1] ))

      trajectories.append(T.Trajectory(trajectory))

We need to store all possible terminal states from the trajectories list. (Needed to calculate the normalizing constant in MaxEnt)

In [ ]:
terminal_states = []

for traj in trajectories:
  terminal_states.append(traj._t[-1][-1])

terminal_states = list(set(terminal_states))

In [ ]:
terminal_states

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

# Distribution of the treatments given in our data. (Most of the time no treatment is given, might vary on depending on how you cluster the data)

In [ ]:
y_df.value_counts()

0.0    195786
1.0    135305
3.0     15978
2.0      8435
Name: action, dtype: int64

# Estimating the Transition Dynamics using the MLE (feel free to play around with the smoothing_value)

In [ ]:
state_encoder = OneHotEncoder(sparse=False, categories= [np.arange(num_clusters)])
action_encoder = OneHotEncoder(sparse=False, categories= [np.arange(action_bound)])

states_onehot = state_encoder.fit_transform(X_df['cluster'].to_numpy().reshape(-1, 1))
actions_onehot = action_encoder.fit_transform(y_df.to_numpy().reshape(-1, 1))

In [ ]:


p_transition = np.zeros((num_clusters, num_clusters, action_bound)) + smoothing_value


for traj in trajectories:

  for tran in traj._t:

    p_transition[tran[0], tran[2], tran[1]] +=1

p_transition = p_transition/ p_transition.sum(axis = 1)[:, np.newaxis, :]

array([[[2.73511648e-01, 2.47170935e-01, 1.79856115e-01, 9.66386555e-02],
        [8.62812770e-04, 5.95592615e-04, 3.59712230e-03, 4.20168067e-03],
        [8.34052344e-03, 7.14711138e-03, 1.43884892e-02, 8.40336134e-03],
        ...,
        [2.87604257e-04, 5.95592615e-04, 3.59712230e-03, 4.20168067e-03],
        [2.87604257e-04, 5.95592615e-04, 3.59712230e-03, 4.20168067e-03],
        [5.75208513e-04, 1.78677784e-03, 3.59712230e-03, 4.20168067e-03]],

       [[2.13675214e-03, 2.53807107e-03, 8.47457627e-03, 7.57575758e-03],
        [1.56695157e-01, 8.88324873e-02, 8.47457627e-03, 1.51515152e-02],
        [8.54700855e-03, 7.61421320e-03, 8.47457627e-03, 7.57575758e-03],
        ...,
        [1.42450142e-03, 2.53807107e-03, 8.47457627e-03, 7.57575758e-03],
        [3.56125356e-03, 5.07614213e-03, 8.47457627e-03, 7.57575758e-03],
        [6.41025641e-03, 1.52284264e-02, 8.47457627e-03, 7.57575758e-03]],

       [[6.45855759e-03, 3.78266850e-03, 1.83150183e-02, 6.12244898e-03],
        

# Max Causal Entropy

# MaxEnt IRL

In [ ]:
"""
Maximum Entropy Inverse Reinforcement Learning and Maximum Causal Entropy
Inverse Reinforcement Learning.

Based on the corresponding paper by B. Ziebart et al. (2008) and the Thesis
by Ziebart (2010).
"""

import numpy as np
from itertools import product


# -- common functions ----------------------------------------------------------

def feature_expectation_from_trajectories(features, trajectories):
    """
    Compute the feature expectation of the given trajectories.

    Simply counts the number of visitations to each feature-instance and
    divides them by the number of trajectories.

    Args:
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        trajectories: A list or iterator of `Trajectory` instances.

    Returns:
        The feature-expectation of the provided trajectories as map
        `[state: Integer] -> feature_expectation: Float`.
    """
    n_states, n_features = features.shape

    fe = np.zeros(n_features)

    for t in trajectories:
        for s in t.states():
            fe += features[s, :]

    return fe / len(trajectories)


def initial_probabilities_from_trajectories(n_states, trajectories):
    """
    Compute the probability of a state being a starting state using the
    given trajectories.

    Args:
        n_states: The number of states.
        trajectories: A list or iterator of `Trajectory` instances.

    Returns:
        The probability of a state being a starting-state as map
        `[state: Integer] -> probability: Float`.
    """
    p = np.zeros(n_states)

    for t in trajectories:
        p[t.transitions()[0][0]] += 1.0

    return p / len(trajectories)


def expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps=1e-5):
    """
    Compute the expected state visitation frequency using the given local
    action probabilities.

    This is the forward pass of Algorithm 1 of the Maximum Entropy IRL paper
    by Ziebart et al. (2008). Alternatively, it can also be found as
    Algorithm 9.3 in in Ziebart's thesis (2010).

    It has been slightly adapted for convergence, by forcing transition
    probabilities from terminal stats to be zero.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: A list of terminal states.
        p_action: Local action probabilities as map
            `[state: Integer, action: Integer] -> probability: Float`
            as returned by `local_action_probabilities`.
        eps: The threshold to be used as convergence criterion. Convergence
            is assumed if the expected state visitation frequency changes
            less than the threshold on all states in a single iteration.

    Returns:
        The expected state visitation frequencies as map
        `[state: Integer] -> svf: Float`.
    """
    n_states, _, n_actions = p_transition.shape

    # 'fix' our transition probabilities to allow for convergence
    # we will _never_ leave any terminal state
    p_transition = np.copy(p_transition)
    p_transition[terminal, :, :] = 0.0

    # set-up transition matrices for each action
    p_transition = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # actual forward-computation of state expectations
    d = np.zeros(n_states)

    delta = np.inf
    while delta > eps:
        d_ = [p_transition[a].T.dot(p_action[:, a] * d) for a in range(n_actions)]
        d_ = p_initial + np.array(d_).sum(axis=0)

        delta, d = np.max(np.abs(d_ - d)), d_

    return d


# -- plain maximum entropy (Ziebart et al. 2008) -------------------------------

def local_action_probabilities(p_transition, terminal, reward):
    """
    Compute the local action probabilities (policy) required for the edge
    frequency calculation for maximum entropy reinfocement learning.

    This is the backward pass of Algorithm 1 of the Maximum Entropy IRL
    paper by Ziebart et al. (2008).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        terminal: A set/list of terminal states.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.

    Returns:
        The local action probabilities (policy) as map
        `[state: Integer, action: Integer] -> probability: Float`
    """
    n_states, _, n_actions = p_transition.shape

    er = np.exp(reward)
    p = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # initialize at terminal states
    zs = np.zeros(n_states)
    zs[terminal] = 1.0

    # perform backward pass
    # This does not converge, instead we iterate a fixed number of steps. The
    # number of steps is chosen to reflect the maximum steps required to
    # guarantee propagation from any state to any other state and back in an
    # arbitrary MDP defined by p_transition.
    for _ in range(2 * n_states):
        za = np.array([er * p[a].dot(zs) for a in range(n_actions)]).T
        zs = za.sum(axis=1)

    # compute local action probabilities
    return za / zs[:, None]


def compute_expected_svf(p_transition, p_initial, terminal, reward, eps=1e-5):
    """
    Compute the expected state visitation frequency for maximum entropy IRL.

    This is an implementation of Algorithm 1 of the Maximum Entropy IRL
    paper by Ziebart et al. (2008).

    This function combines the backward pass implemented in
    `local_action_probabilities` with the forward pass implemented in
    `expected_svf_from_policy`.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: A list of terminal states.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        eps: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.

    Returns:
        The expected state visitation frequencies as map
        `[state: Integer] -> svf: Float`.
    """
    p_action = local_action_probabilities(p_transition, terminal, reward)
    return expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps)


def irl(p_transition, features, terminal, trajectories, optim, init, eps=1e-4, eps_esvf=1e-5):
    """
    Compute the reward signal given the demonstration trajectories using the
    maximum entropy inverse reinforcement learning algorithm proposed in the
    corresponding paper by Ziebart et al. (2008).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        terminal: A list of terminal states.
        trajectories: A list of `Trajectory` instances representing the
            expert demonstrations.
        optim: The `Optimizer` instance to use for gradient-based
            optimization.
        init: The `Initializer` to use for initialization of the reward
            function parameters.
        eps: The threshold to be used as convergence criterion for the
            reward parameters. Convergence is assumed if all changes in the
            scalar parameters are less than the threshold in a single
            iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.

    Returns:
        The reward per state as table `[state: Integer] -> reward: Float`.
    """
    n_states, _, n_actions = p_transition.shape
    _, n_features = features.shape

    # compute static properties from trajectories
    e_features = feature_expectation_from_trajectories(features, trajectories)
    p_initial = initial_probabilities_from_trajectories(n_states, trajectories)

    # basic gradient descent
    theta = init(n_features)
    delta = np.inf

    optim.reset(theta)
    count = 0
    while delta > eps:
        if count > 10000:
            print('diverges: max ent')
            break
        theta_old = theta.copy()

        # compute per-state reward
        reward = features.dot(theta)

        # compute the gradient
        e_svf = compute_expected_svf(p_transition, p_initial, terminal, reward, eps_esvf)
        grad = e_features - features.T.dot(e_svf)

        # perform optimization step and compute delta for convergence
        optim.step(grad)
        delta = np.max(np.abs(theta_old - theta))
        count += 1

    # re-compute per-state reward and return
    return features.dot(theta)


# -- maximum causal entropy (Ziebart 2010) -------------------------------------

def softmax(x1, x2):
    """
    Computes a soft maximum of both arguments.

    In case `x1` and `x2` are arrays, computes the element-wise softmax.

    Args:
        x1: Scalar or ndarray.
        x2: Scalar or ndarray.

    Returns:
        The soft maximum of the given arguments, either scalar or ndarray,
        depending on the input.
    """
    x_max = np.maximum(x1, x2)
    x_min = np.minimum(x1, x2)
    return x_max + np.log(1.0 + np.exp(x_min - x_max))


def local_causal_action_probabilities(p_transition, terminal, reward, discount, eps=1e-5):
    """
    Compute the local action probabilities (policy) required for the edge
    frequency calculation for maximum causal entropy reinfocement learning.

    This is Algorithm 9.1 from Ziebart's thesis (2010) combined with
    discounting for convergence reasons as proposed in the same thesis.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        discount: A discounting factor as Float.
        eps: The threshold to be used as convergence criterion for the state
            partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.

    Returns:
        The local action probabilities (policy) as map
        `[state: Integer, action: Integer] -> probability: Float`
    """
    n_states, _, n_actions = p_transition.shape

    # set up terminal reward function
    if len(terminal) == n_states:
        reward_terminal = np.array(terminal, dtype=float)
    else:
        reward_terminal = -np.inf * np.ones(n_states)
        reward_terminal[terminal] = 0.0

    # set up transition probability matrices
    p = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # compute state log partition V and state-action log partition Q
    v = -1e200 * np.ones(n_states)  # np.dot doesn't behave with -np.inf

    delta = np.inf
    while delta > eps:
        v_old = v

        q = np.array([reward + discount * p[a].dot(v_old) for a in range(n_actions)]).T

        v = reward_terminal
        for a in range(n_actions):
            v = softmax(v, q[:, a])

        # for some reason numpy chooses an array of objects after reduction, force floats here
        v = np.array(v, dtype=float)

        delta = np.max(np.abs(v - v_old))

    # compute and return policy
    return np.exp(q - v[:, None])


def compute_expected_causal_svf(p_transition, p_initial, terminal, reward, discount,
                                eps_lap=1e-5, eps_svf=1e-5):
    """
    Compute the expected state visitation frequency for maximum causal
    entropy IRL.

    This is a combination of Algorithm 9.1 and 9.3 of Ziebart's thesis
    (2010). See `local_causal_action_probabilities` and
    `expected_svf_from_policy` for more details.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        discount: A discounting factor as Float.
        eps_lap: The threshold to be used as convergence criterion for the
            state partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.
    """
    p_action = local_causal_action_probabilities(p_transition, terminal, reward, discount, eps_lap)
    return expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps_svf)


def irl_causal(p_transition, features, terminal, trajectories, optim, init, discount,
               eps=1e-4, eps_svf=1e-5, eps_lap=1e-5):
    """
    Compute the reward signal given the demonstration trajectories using the
    maximum causal entropy inverse reinforcement learning algorithm proposed
    Ziebart's thesis (2010).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        trajectories: A list of `Trajectory` instances representing the
            expert demonstrations.
        optim: The `Optimizer` instance to use for gradient-based
            optimization.
        init: The `Initializer` to use for initialization of the reward
            function parameters.
        discount: A discounting factor for the log partition functions as
            Float.
        eps: The threshold to be used as convergence criterion for the
            reward parameters. Convergence is assumed if all changes in the
            scalar parameters are less than the threshold in a single
            iteration.
        eps_lap: The threshold to be used as convergence criterion for the
            state partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.
    """
    n_states, _, n_actions = p_transition.shape
    _, n_features = features.shape

    # compute static properties from trajectories
    e_features = feature_expectation_from_trajectories(features, trajectories)
    p_initial = initial_probabilities_from_trajectories(n_states, trajectories)

    # basic gradient descent
    theta = init(n_features)
    delta = np.inf

    optim.reset(theta)

    count = 0
    while delta > eps:
        if count > 10000:
            print('diverges: max causal')
            break
        theta_old = theta.copy()

        # compute per-state reward
        reward = features.dot(theta)

        # compute the gradient
        e_svf = compute_expected_causal_svf(p_transition, p_initial, terminal, reward, discount,
                                            eps_lap, eps_svf)

        grad = e_features - features.T.dot(e_svf)

        # perform optimization step and compute delta for convergence
        optim.step(grad)
        delta = np.max(np.abs(theta_old - theta))

        if count % 50 == 0:
            print(delta)

        count += 1

    # re-compute per-state reward and return
    return features.dot(theta)

# Max CAUSAL IRL

In [ ]:
"""
Maximum Entropy Inverse Reinforcement Learning and Maximum Causal Entropy
Inverse Reinforcement Learning.

Based on the corresponding paper by B. Ziebart et al. (2008) and the Thesis
by Ziebart (2010).
"""

import numpy as np
from itertools import product


# -- common functions ----------------------------------------------------------

def feature_expectation_from_trajectories(features, trajectories):
    """
    Compute the feature expectation of the given trajectories.

    Simply counts the number of visitations to each feature-instance and
    divides them by the number of trajectories.

    Args:
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        trajectories: A list or iterator of `Trajectory` instances.

    Returns:
        The feature-expectation of the provided trajectories as map
        `[state: Integer] -> feature_expectation: Float`.
    """
    n_states, n_features = features.shape

    fe = np.zeros(n_features)

    for t in trajectories:
        for s in t.states():
            fe += features[s, :]

    return fe / len(trajectories)


def initial_probabilities_from_trajectories(n_states, trajectories):
    """
    Compute the probability of a state being a starting state using the
    given trajectories.

    Args:
        n_states: The number of states.
        trajectories: A list or iterator of `Trajectory` instances.

    Returns:
        The probability of a state being a starting-state as map
        `[state: Integer] -> probability: Float`.
    """
    p = np.zeros(n_states)

    for t in trajectories:
        p[t.transitions()[0][0]] += 1.0

    return p / len(trajectories)


def expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps=1e-5):
    """
    Compute the expected state visitation frequency using the given local
    action probabilities.

    This is the forward pass of Algorithm 1 of the Maximum Entropy IRL paper
    by Ziebart et al. (2008). Alternatively, it can also be found as
    Algorithm 9.3 in in Ziebart's thesis (2010).

    It has been slightly adapted for convergence, by forcing transition
    probabilities from terminal stats to be zero.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: A list of terminal states.
        p_action: Local action probabilities as map
            `[state: Integer, action: Integer] -> probability: Float`
            as returned by `local_action_probabilities`.
        eps: The threshold to be used as convergence criterion. Convergence
            is assumed if the expected state visitation frequency changes
            less than the threshold on all states in a single iteration.

    Returns:
        The expected state visitation frequencies as map
        `[state: Integer] -> svf: Float`.
    """
    n_states, _, n_actions = p_transition.shape

    # 'fix' our transition probabilities to allow for convergence
    # we will _never_ leave any terminal state
    p_transition = np.copy(p_transition)
    p_transition[terminal, :, :] = 0.0

    # set-up transition matrices for each action
    p_transition = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # actual forward-computation of state expectations
    d = np.zeros(n_states)

    delta = np.inf
    while delta > eps:
        d_ = [p_transition[a].T.dot(p_action[:, a] * d) for a in range(n_actions)]
        d_ = p_initial + np.array(d_).sum(axis=0)

        delta, d = np.max(np.abs(d_ - d)), d_

    return d


# -- plain maximum entropy (Ziebart et al. 2008) -------------------------------

def local_action_probabilities(p_transition, terminal, reward):
    """
    Compute the local action probabilities (policy) required for the edge
    frequency calculation for maximum entropy reinfocement learning.

    This is the backward pass of Algorithm 1 of the Maximum Entropy IRL
    paper by Ziebart et al. (2008).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        terminal: A set/list of terminal states.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.

    Returns:
        The local action probabilities (policy) as map
        `[state: Integer, action: Integer] -> probability: Float`
    """
    n_states, _, n_actions = p_transition.shape

    er = np.exp(reward)
    p = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # initialize at terminal states
    zs = np.zeros(n_states)
    zs[terminal] = 1.0

    # perform backward pass
    # This does not converge, instead we iterate a fixed number of steps. The
    # number of steps is chosen to reflect the maximum steps required to
    # guarantee propagation from any state to any other state and back in an
    # arbitrary MDP defined by p_transition.
    for _ in range(2 * n_states):
        za = np.array([er * p[a].dot(zs) for a in range(n_actions)]).T
        zs = za.sum(axis=1)

    # compute local action probabilities
    return za / zs[:, None]


def compute_expected_svf(p_transition, p_initial, terminal, reward, eps=1e-5):
    """
    Compute the expected state visitation frequency for maximum entropy IRL.

    This is an implementation of Algorithm 1 of the Maximum Entropy IRL
    paper by Ziebart et al. (2008).

    This function combines the backward pass implemented in
    `local_action_probabilities` with the forward pass implemented in
    `expected_svf_from_policy`.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: A list of terminal states.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        eps: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.

    Returns:
        The expected state visitation frequencies as map
        `[state: Integer] -> svf: Float`.
    """
    p_action = local_action_probabilities(p_transition, terminal, reward)
    return expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps)


def irl(p_transition, features, terminal, trajectories, optim, init, eps=1e-4, eps_esvf=1e-5):
    """
    Compute the reward signal given the demonstration trajectories using the
    maximum entropy inverse reinforcement learning algorithm proposed in the
    corresponding paper by Ziebart et al. (2008).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        terminal: A list of terminal states.
        trajectories: A list of `Trajectory` instances representing the
            expert demonstrations.
        optim: The `Optimizer` instance to use for gradient-based
            optimization.
        init: The `Initializer` to use for initialization of the reward
            function parameters.
        eps: The threshold to be used as convergence criterion for the
            reward parameters. Convergence is assumed if all changes in the
            scalar parameters are less than the threshold in a single
            iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.

    Returns:
        The reward per state as table `[state: Integer] -> reward: Float`.
    """
    n_states, _, n_actions = p_transition.shape
    _, n_features = features.shape

    # compute static properties from trajectories
    e_features = feature_expectation_from_trajectories(features, trajectories)
    p_initial = initial_probabilities_from_trajectories(n_states, trajectories)

    # basic gradient descent
    theta = init(n_features)
    delta = np.inf

    optim.reset(theta)
    while delta > eps:
        theta_old = theta.copy()

        # compute per-state reward
        reward = features.dot(theta)

        # compute the gradient
        e_svf = compute_expected_svf(p_transition, p_initial, terminal, reward, eps_esvf)
        grad = e_features - features.T.dot(e_svf)

        # perform optimization step and compute delta for convergence
        optim.step(grad)
        delta = np.max(np.abs(theta_old - theta))

    # re-compute per-state reward and return
    return features.dot(theta)


# -- maximum causal entropy (Ziebart 2010) -------------------------------------

def softmax(x1, x2):
    """
    Computes a soft maximum of both arguments.

    In case `x1` and `x2` are arrays, computes the element-wise softmax.

    Args:
        x1: Scalar or ndarray.
        x2: Scalar or ndarray.

    Returns:
        The soft maximum of the given arguments, either scalar or ndarray,
        depending on the input.
    """
    x_max = np.maximum(x1, x2)
    x_min = np.minimum(x1, x2)
    return x_max + np.log(1.0 + np.exp(x_min - x_max))


def local_causal_action_probabilities(p_transition, terminal, reward, discount, eps=1e-5):
    """
    Compute the local action probabilities (policy) required for the edge
    frequency calculation for maximum causal entropy reinfocement learning.

    This is Algorithm 9.1 from Ziebart's thesis (2010) combined with
    discounting for convergence reasons as proposed in the same thesis.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        discount: A discounting factor as Float.
        eps: The threshold to be used as convergence criterion for the state
            partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.

    Returns:
        The local action probabilities (policy) as map
        `[state: Integer, action: Integer] -> probability: Float`
    """
    n_states, _, n_actions = p_transition.shape

    # set up terminal reward function
    if len(terminal) == n_states:
        reward_terminal = np.array(terminal, dtype=float)
    else:
        reward_terminal = -np.inf * np.ones(n_states)
        reward_terminal[terminal] = 0.0

    # set up transition probability matrices
    p = [np.array(p_transition[:, :, a]) for a in range(n_actions)]

    # compute state log partition V and state-action log partition Q
    v = -1e200 * np.ones(n_states)  # np.dot doesn't behave with -np.inf

    delta = np.inf
    while delta > eps:
        v_old = v

        q = np.array([reward + discount * p[a].dot(v_old) for a in range(n_actions)]).T

        v = reward_terminal
        for a in range(n_actions):
            v = softmax(v, q[:, a])

        # for some reason numpy chooses an array of objects after reduction, force floats here
        v = np.array(v, dtype=float)

        delta = np.max(np.abs(v - v_old))

    # compute and return policy
    return np.exp(q - v[:, None])


def compute_expected_causal_svf(p_transition, p_initial, terminal, reward, discount,
                                eps_lap=1e-5, eps_svf=1e-5):
    """
    Compute the expected state visitation frequency for maximum causal
    entropy IRL.

    This is a combination of Algorithm 9.1 and 9.3 of Ziebart's thesis
    (2010). See `local_causal_action_probabilities` and
    `expected_svf_from_policy` for more details.

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        p_initial: The probability of a state being an initial state as map
            `[state: Integer] -> probability: Float`.
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        reward: The reward signal per state as table
            `[state: Integer] -> reward: Float`.
        discount: A discounting factor as Float.
        eps_lap: The threshold to be used as convergence criterion for the
            state partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.
    """
    p_action = local_causal_action_probabilities(p_transition, terminal, reward, discount, eps_lap)
    return expected_svf_from_policy(p_transition, p_initial, terminal, p_action, eps_svf)


def irl_causal(X_df, p_transition, features, terminal, trajectories, optim, init, discount,eps=1e-4, eps_svf=1e-5, eps_lap=1e-5):
    """
    Compute the reward signal given the demonstration trajectories using the
    maximum causal entropy inverse reinforcement learning algorithm proposed
    Ziebart's thesis (2010).

    Args:
        p_transition: The transition probabilities of the MDP as table
            `[from: Integer, to: Integer, action: Integer] -> probability: Float`
            specifying the probability of a transition from state `from` to
            state `to` via action `action` to succeed.
        features: The feature-matrix (e.g. as numpy array), mapping states
            to features, i.e. a matrix of shape (n_states x n_features).
        terminal: Either the terminal reward function or a collection of
            terminal states. Iff `len(terminal)` is equal to the number of
            states, it is assumed to contain the terminal reward function
            (phi) as specified in Ziebart's thesis. Otherwise `terminal` is
            assumed to be a collection of terminal states from which the
            terminal reward function will be derived.
        trajectories: A list of `Trajectory` instances representing the
            expert demonstrations.
        optim: The `Optimizer` instance to use for gradient-based
            optimization.
        init: The `Initializer` to use for initialization of the reward
            function parameters.
        discount: A discounting factor for the log partition functions as
            Float.
        eps: The threshold to be used as convergence criterion for the
            reward parameters. Convergence is assumed if all changes in the
            scalar parameters are less than the threshold in a single
            iteration.
        eps_lap: The threshold to be used as convergence criterion for the
            state partition function. Convergence is assumed if the state
            partition function changes less than the threshold on all states
            in a single iteration.
        eps_svf: The threshold to be used as convergence criterion for the
            expected state-visitation frequency. Convergence is assumed if
            the expected state visitation frequency changes less than the
            threshold on all states in a single iteration.
    """
    n_states, _, n_actions = p_transition.shape
    _, n_features = features.shape

    # compute static properties from trajectories
    e_features = feature_expectation_from_trajectories(features, trajectories)
    p_initial = initial_probabilities_from_trajectories(n_states, trajectories)

    # basic gradient descent
    theta = init(n_features)
    delta = np.inf

    # input data
    epochs_infos = []

    optim.reset(theta)
    count = 0
    while delta > eps:
        if count > 10000:
            print("does not converge")
            break

        theta_old = theta.copy()

        # compute per-state reward
        reward = features.dot(theta)

        epochs_infos.append(reward.tolist())

        # compute the gradient
        e_svf = compute_expected_causal_svf(p_transition, p_initial, terminal, reward, discount,
                                            eps_lap, eps_svf)

        grad = e_features - features.T.dot(e_svf)

        # perform optimization step and compute delta for convergence
        optim.step(grad)
        delta = np.max(np.abs(theta_old - theta))
        count +=1

    # re-compute per-state reward and return
    final_reward =features.dot(theta)
    epochs_infos.append(final_reward.tolist())

    return epochs_infos, final_reward

def update(reward_list, X_df):

    rankings = list(np.argsort(reward_list))
    # lowest to highest
    all_cluster_info = {}
    for i in range(len(reward_list)):
        # reward_maxent_causal[i] is the reward value
        cluster_id = i+1
        reward_value = reward_list[i]
        data = X_df[(X_df['cluster'] == i+1)]
        all_cluster_info[cluster_id] = data.mean().to_dict()
        all_cluster_info[cluster_id]['reward_value'] = reward_value
        # note: rankings are 0-indexed with the unhealthiest first (lowest)
        all_cluster_info[cluster_id]['ranking'] = rankings.index(i)
    # add to history
    return all_cluster_info

def extract_values(discount_val, features, cluster_algo, smooth_val, learning_rate=0.2, data=''):
  # choose our parameter initialization strategy:
  #   initialize parameters with constant
  init = O.Constant(1.0)

  # choose our optimization strategy:
  #   we select exponentiated stochastic gradient descent with linear learning-rate decay
  optim = O.ExpSga(lr=O.linear_decay(lr0=learning_rate))

  # actually do some inverse reinforcement learning
  # reward_maxent = maxent_irl(p_transition, features, terminal_states, trajectories, optim, init, eps= 1e-3)

  # send in the X_df dataframe
  epochs_infos, final_reward = irl_causal(data_non_normalized_df, p_transition, features, terminal_states, trajectories, optim, init, discount_val, eps=1e-3, eps_svf=1e-4, eps_lap=1e-4)

  if data != '':
    file_name = data+'_mce_' + cluster_algo + str(discount_val)+'_smooth'+str(smooth_val)+'_lr'+str(learning_rate)+'.json'
  else:
    file_name = '_mce_' + cluster_algo + str(discount_val)+'_smooth'+str(smooth_val)+'.json'

  final_cluster_info = update(final_reward, data_non_normalized_df)
  print("----")
  print(final_cluster_info)
  print("----")
  # print(epochs_infos)
  print("---")
  print(np.array(epochs_infos).shape)
  with open(file_name, 'w') as m:
    json.dump(epochs_infos, m)

  return epochs_infos

In [ ]:
epoch_reward_values = extract_values(discount_exp, features, 'kmeans', smoothing_val_exp, learning_rate=0.1, data='pneumonia')
